In [4]:
import re
import xarray
import requests
import warnings
import matplotlib
import numpy as np
import holoviews as hv
from matplotlib import rc
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from matplotlib.animation import FuncAnimation

In [1]:
# define the url of the thredds server
thredds_base='http://thredds.hydroshare.org/thredds'

In [2]:
# specify date and forcast arguments
date = '20170910'
configuration = 'medium_range'
output_type = 'land'

# build the full catalog url and define the namespace for XML parsing
catalog = '%s/catalog/nwm/harvey/nwm.%s/%s/catalog.xml' % (thredds_base, date, configuration)
namespaces = {'unidata': 'http://www.unidata.ucar.edu/namespaces/thredds/InvCatalog/v1.0'}

In [5]:
r = requests.get(catalog)
tree = ET.ElementTree(ET.fromstring(r.text))
root = tree.getroot()

In [6]:
# filter to our data of interest
unidata_datasets = root.findall('unidata:dataset/unidata:dataset', namespaces)
pattern = "^.*t00z.%s.%s.*$" % (configuration, output_type)
paths = [] 
for ds in unidata_datasets:
    name = ds.attrib['name']
    if re.match(pattern, name) is not None:
        paths.append('/'.join([thredds_base, 'dodsC', ds.attrib['urlPath']]))

In [7]:
single_dataset = xarray.open_dataset(paths[10])

In [8]:
single_dataset

<xarray.Dataset>
Dimensions:                     (reference_time: 1, snow_layers: 3, soil_layers_stag: 4, time: 1, x: 4608, y: 3840)
Coordinates:
  * time                        (time) datetime64[ns] 2017-09-18T18:00:00
  * x                           (x) float64 -2.304e+06 -2.303e+06 ... 2.303e+06
  * y                           (y) float64 -1.92e+06 -1.919e+06 ... 1.919e+06
  * reference_time              (reference_time) datetime64[ns] 2017-09-10
Dimensions without coordinates: snow_layers, soil_layers_stag
Data variables:
    SNLIQ                       (time, y, snow_layers, x) float64 ...
    SOIL_T                      (time, y, soil_layers_stag, x) float64 ...
    SOIL_M                      (time, y, soil_layers_stag, x) float64 ...
    ProjectionCoordinateSystem  |S64 ...
    FSA                         (time, y, x) float64 ...
    FIRA                        (time, y, x) float64 ...
    GRDFLX                      (time, y, x) float64 ...
    HFX                         (time, y, x) float64 ...
    LH                          (time, y, x) float64 ...
    UGDRNOFF                    (time, y, x) float64 ...
    ACCECAN                     (time, y, x) float64 ...
    ACCEDIR                     (time, y, x) float64 ...
    ACCETRAN                    (time, y, x) float64 ...
    TRAD                        (time, y, x) float64 ...
    SNOWH                       (time, y, x) float64 ...
    SNEQV                       (time, y, x) float64 ...
    ISNOW                       (time, y, x) float64 ...
    FSNO                        (time, y, x) float64 ...
    ACSNOM                      (time, y, x) float64 ...
    ACCET                       (time, y, x) float64 ...
    CANWAT                      (time, y, x) float64 ...
    SOILICE                     (time, y, x) float64 ...
    SOILSAT_TOP                 (time, y, x) float64 ...
    SNOWT_AVG                   (time, y, x) float64 ...
Attributes:
    TITLE:                           OUTPUT FROM HRLDAS v20150506
    model_initialization_time:       2017-09-10_00:00:00
    model_output_valid_time:         2017-09-18_18:00:00
    Conventions:                     CF-1.6
    DODS.strlen:                     0
    DODS_EXTRA.Unlimited_Dimension:  time

In [9]:
mds = xarray.open_mfdataset(paths)

In [10]:
mds

,Array,Chunk
Bytes,33.97 GB,424.67 MB
Shape,"(80, 3840, 3, 4608)","(1, 3840, 3, 4608)"
Count,240 Tasks,80 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,45.30 GB,566.23 MB
Shape,"(80, 3840, 4, 4608)","(1, 3840, 4, 4608)"
Count,240 Tasks,80 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [11]:
lon_max = -770000.0
lon_min = -1060000.0
lat_max = 218000.0
lat_min = -5000.0

In [12]:
lats = mds.variables['y'][:] 
lons = mds.variables['x'][:]
lat_bnds = [ lat_min , lat_max ]
lon_bnds = [ lon_min , lon_max ] 

lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))
lon_inds = np.where((lons > lon_bnds[0]) & (lons < lon_bnds[1]))

In [13]:
llat = lat_inds[0].min()
ulat = lat_inds[0].max()
llon = lon_inds[0].min()
ulon = lon_inds[0].max()

print('latitude index range:  %d -> %d' % (llat, ulat))
print('longitude index range: %d -> %d' % (llon, ulon))

latitude index range:  1915 -> 2137
longitude index range: 1244 -> 1533


In [14]:
# perform subset
subset = mds.isel(x=lon_inds[0], y=lat_inds[0])

# view the subsetted data
subset

,Array,Chunk
Bytes,124.17 MB,1.55 MB
Shape,"(80, 223, 3, 290)","(1, 223, 3, 290)"
Count,400 Tasks,80 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,165.56 MB,2.07 MB
Shape,"(80, 223, 4, 290)","(1, 223, 4, 290)"
Count,400 Tasks,80 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
